In [1]:
!pip install fastapi uvicorn
!pip install -U langchain-community

In [5]:
# ✅ Step 4: LangGraph + LangChain Interview Flow (Final Fixed Version)
# --------------------------------------------------------------------
import os
import logging
from typing import Optional
from dotenv import load_dotenv
from pydantic import BaseModel
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, END

# Load environment variables
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# -------------------------------------------------------
# 🧠 Initialize LLM
# -------------------------------------------------------
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7, api_key=api_key)

# -------------------------------------------------------
# 🧩 Define Interview State
# -------------------------------------------------------
class InterviewState(BaseModel):
    round_type: Optional[str] = None
    context: Optional[str] = None
    question: Optional[str] = None
    candidate_answer: Optional[str] = None
    evaluation: Optional[str] = None

# -------------------------------------------------------
# 🔧 Define Chains (using RunnableSequence)
# -------------------------------------------------------

# Question generation
question_prompt = ChatPromptTemplate.from_template(
    "Generate one interview question for a {round_type} round based on this context: {context}"
)
question_chain = question_prompt | llm

# Answer evaluation
evaluation_prompt = ChatPromptTemplate.from_template(
    "Evaluate this candidate answer for the question '{question}': {candidate_answer}. "
    "Give a brief evaluation summary."
)
evaluation_chain = evaluation_prompt | llm

# -------------------------------------------------------
# 🧱 Define Node Functions
# -------------------------------------------------------

def generate_question(state: InterviewState) -> InterviewState:
    logging.info("🎯 Generating interview question...")
    result = question_chain.invoke({
        "round_type": state.round_type or "General",
        "context": state.context or "software engineering interview"
    })

    # Extract text safely
    question_text = getattr(result, "content", str(result)).strip()

    # Update state correctly (avoid duplicate keyword)
    state_data = state.model_dump()
    state_data["question"] = question_text

    return InterviewState(**state_data)

def evaluate_response(state: InterviewState) -> InterviewState:
    logging.info("🧩 Evaluating candidate's response...")
    result = evaluation_chain.invoke({
        "question": state.question,
        "candidate_answer": state.candidate_answer
    })

    evaluation_text = getattr(result, "content", str(result)).strip()
    state_data = state.model_dump()
    state_data["evaluation"] = evaluation_text

    return InterviewState(**state_data)

# -------------------------------------------------------
# 🕸️ Build LangGraph
# -------------------------------------------------------
graph = StateGraph(InterviewState)
graph.add_node("generate_question", generate_question)
graph.add_node("evaluate_response", evaluate_response)

graph.set_entry_point("generate_question")
graph.add_edge("generate_question", "evaluate_response")
graph.add_edge("evaluate_response", END)

interview_graph = graph.compile()

# -------------------------------------------------------
# 🚀 Run Flow
# -------------------------------------------------------
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

logger.info("🚀 Running Step-4: LangGraph Interview Flow")

init_state = InterviewState(
    round_type="HR",
    context="Candidate is a software engineer with 2 years of experience."
)

# 1️⃣ Generate Question
result_1 = interview_graph.invoke(init_state)
state_after_q = InterviewState(**result_1)

print(f"\n🧠 Asked Question: {state_after_q.question}")

# 2️⃣ Add Candidate's Answer
state_after_q.candidate_answer = (
    "I once handled a conflict by organizing a team meeting "
    "and discussing our goals openly to find a shared solution."
)

# 3️⃣ Evaluate
result_2 = interview_graph.invoke(state_after_q)
final_state = InterviewState(**result_2)

print(f"\n💬 Candidate Answer: {final_state.candidate_answer}")
print(f"\n📊 Evaluation: {final_state.evaluation}")


INFO:__main__:🚀 Running Step-4: LangGraph Interview Flow
INFO:root:🎯 Generating interview question...
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:🧩 Evaluating candidate's response...
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:🎯 Generating interview question...



🧠 Asked Question: Can you describe a challenging project you worked on in the past two years and how you approached problem-solving during that project?


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:🧩 Evaluating candidate's response...
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



💬 Candidate Answer: I once handled a conflict by organizing a team meeting and discussing our goals openly to find a shared solution.

📊 Evaluation: The candidate's answer provides a glimpse into their approach to conflict resolution within a team setting, which is a valuable skill in software engineering. However, the response lacks specific details about the challenging project itself, such as the project's nature, the obstacles faced, and the technical aspects involved. The mention of organizing a team meeting is a good start, but it would be more impactful if the candidate elaborated on the challenges they encountered and how their actions directly contributed to overcoming those challenges. Overall, the answer could be strengthened by providing more context and demonstrating a deeper involvement in the project's technical aspects and outcomes.


# 🕸️ LangGraph Advanced Examples

This notebook demonstrates advanced LangGraph capabilities including:

1. **Multi-step Workflows** - Complex interview processes with multiple decision points
2. **Conditional Logic** - Dynamic workflow paths based on candidate responses
3. **State Persistence** - Memory management across multiple interactions
4. **Error Handling** - Robust error handling and fallback mechanisms
5. **Custom Node Functions** - Specialized processing for different interview types


In [ ]:
# 🚀 Advanced LangGraph Implementation
# Import the new LangGraph processor from the project
import sys
import os
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(''))))

from ai_modules.langgraph_processor import (
    LangGraphInterviewProcessor, 
    create_interview_processor,
    InterviewState
)

print("✅ Advanced LangGraph processor imported successfully!")


In [ ]:
# 🎯 Example 1: Multi-Round Interview Workflow
print("🎯 Example 1: Multi-Round Interview Workflow")
print("=" * 50)

# Create processor
processor = create_interview_processor(use_openai=True)

# Define interview scenarios
interview_scenarios = [
    {
        "round": "HR",
        "context": "Senior software engineer with 5 years experience",
        "answer": "I led a team of 4 developers on a critical project. We used agile methodology and daily standups to ensure smooth communication."
    },
    {
        "round": "Technical", 
        "context": "Full-stack developer position",
        "answer": "For a REST API, I would use Express.js with proper error handling, input validation, and rate limiting. I'd also implement JWT authentication."
    },
    {
        "round": "Behavioral",
        "context": "Team lead role",
        "answer": "When facing a tight deadline, I prioritize tasks based on impact and dependencies, communicate with stakeholders about trade-offs, and ensure quality isn't compromised."
    }
]

# Run multiple rounds
results = []
for i, scenario in enumerate(interview_scenarios, 1):
    print(f"\n🔄 Round {i}: {scenario['round']} Interview")
    print("-" * 30)
    
    result = processor.run_interview_round(
        round_type=scenario["round"],
        context=scenario["context"],
        candidate_answer=scenario["answer"]
    )
    
    results.append(result)
    
    print(f"Question: {result['question']}")
    print(f"Answer: {result['candidate_answer']}")
    print(f"Score: {result['score']}")
    print(f"Success: {result['success']}")

# Show session summary
print(f"\n📊 Session Summary:")
summary = processor.get_session_summary()
print(f"Total Interactions: {summary['total_interactions']}")
print(f"Round Types: {summary['round_types']}")
print(f"Average Score: {summary['average_score']:.1f}")
print(f"Model: {summary['model']}")


In [ ]:
# 🔧 Example 2: Custom State Management
print("\n🔧 Example 2: Custom State Management")
print("=" * 50)

# Create a new processor for this example
processor2 = create_interview_processor(use_openai=True)

# Create custom initial state
custom_state = InterviewState(
    round_type="Technical",
    context="Machine learning engineer with expertise in deep learning",
    metadata={
        "session_id": "demo_001",
        "candidate_level": "senior",
        "specialization": "ML/DL",
        "experience_years": 5
    }
)

print(f"Initial State:")
print(f"  Round Type: {custom_state.round_type}")
print(f"  Context: {custom_state.context}")
print(f"  Metadata: {custom_state.metadata}")

# Run workflow with custom state
result = processor2.run_interview_round(
    round_type=custom_state.round_type,
    context=custom_state.context,
    candidate_answer="I would implement a neural network using TensorFlow, with proper data preprocessing, cross-validation, and hyperparameter tuning. I'd also use techniques like dropout and batch normalization to prevent overfitting."
)

print(f"\nWorkflow Result:")
print(f"  Question: {result['question']}")
print(f"  Score: {result['score']}")
print(f"  Current Step: {result['current_step']}")
print(f"  Success: {result['success']}")

# Show conversation history
if result.get('conversation_history'):
    print(f"\n📝 Conversation History:")
    for i, interaction in enumerate(result['conversation_history'], 1):
        print(f"  Interaction {i}:")
        print(f"    Round: {interaction.get('round_type', 'Unknown')}")
        print(f"    Score: {interaction.get('score', {}).get('overall', 'N/A')}")


In [ ]:
# 🛡️ Example 3: Error Handling and Fallback
print("\n🛡️ Example 3: Error Handling and Fallback")
print("=" * 50)

# Test with fallback mode (no OpenAI)
print("Testing fallback mode (no OpenAI API key)...")

# Temporarily remove API key for testing
original_key = os.environ.get('OPENAI_API_KEY')
if 'OPENAI_API_KEY' in os.environ:
    del os.environ['OPENAI_API_KEY']

# Create processor in fallback mode
fallback_processor = create_interview_processor(use_openai=False)

print(f"Processor created with OpenAI: {fallback_processor.use_openai}")
print(f"Model: {fallback_processor.model}")

# Test fallback functionality
fallback_result = fallback_processor.run_interview_round(
    round_type="HR",
    context="Software engineer",
    candidate_answer="I believe in clear communication and regular team meetings to ensure everyone is aligned on project goals."
)

print(f"\nFallback Result:")
print(f"  Question: {fallback_result['question']}")
print(f"  Answer: {fallback_result['candidate_answer']}")
print(f"  Evaluation: {fallback_result['evaluation']}")
print(f"  Score: {fallback_result['score']}")
print(f"  Success: {fallback_result['success']}")

# Restore API key
if original_key:
    os.environ['OPENAI_API_KEY'] = original_key

print(f"\n✅ Error handling test completed successfully!")


In [ ]:
# 🎨 Example 4: Interactive Interview Simulation
print("\n🎨 Example 4: Interactive Interview Simulation")
print("=" * 50)

def interactive_interview():
    """Interactive interview simulation"""
    processor = create_interview_processor(use_openai=True)
    
    print("🎮 Interactive Interview Simulation")
    print("You can provide your own answers to see how the system evaluates them.")
    print("Type 'quit' to exit the simulation.\n")
    
    round_count = 0
    
    while True:
        round_count += 1
        print(f"\n{'='*40}")
        print(f"Round {round_count}")
        print(f"{'='*40}")
        
        round_type = input("Enter round type (HR/Technical/Behavioral) or 'quit': ").strip()
        
        if round_type.lower() == 'quit':
            break
        
        if round_type not in ['HR', 'Technical', 'Behavioral']:
            print("❌ Invalid round type. Please use HR, Technical, or Behavioral.")
            continue
        
        context = input("Enter candidate context (or press Enter for default): ").strip()
        if not context:
            context = f"Candidate applying for {round_type} position"
        
        # Generate question
        print("\n🤖 Generating question...")
        question_result = processor.run_interview_round(
            round_type=round_type,
            context=context,
            candidate_answer=None
        )
        
        print(f"\n🧠 Question: {question_result['question']}")
        
        # Get user answer
        user_answer = input("\n💬 Your answer: ").strip()
        
        if not user_answer:
            print("❌ No answer provided. Skipping evaluation.")
            continue
        
        # Evaluate answer
        print("\n🤖 Evaluating your answer...")
        evaluation_result = processor.run_interview_round(
            round_type=round_type,
            context=context,
            candidate_answer=user_answer
        )
        
        print(f"\n📊 Evaluation: {evaluation_result['evaluation']}")
        print(f"⭐ Score: {evaluation_result['score']}")
    
    # Show final summary
    print(f"\n📈 Final Session Summary:")
    summary = processor.get_session_summary()
    print(f"Total Interactions: {summary['total_interactions']}")
    print(f"Round Types: {summary['round_types']}")
    print(f"Average Score: {summary['average_score']:.1f}")
    print(f"Model Used: {summary['model']}")
    
    print("\n👋 Thanks for trying the interactive simulation!")

# Uncomment the line below to run interactive simulation
# interactive_interview()

print("💡 To run the interactive simulation, uncomment the last line in this cell!")


# 📚 LangGraph Implementation Summary

## ✅ What We've Implemented

### 1. **Core LangGraph Module** (`ai_modules/langgraph_processor.py`)
- **InterviewState**: Pydantic model for state management
- **LangGraphInterviewProcessor**: Main processor class with workflow orchestration
- **Memory Management**: Conversation history and state persistence
- **Error Handling**: Robust fallback mechanisms

### 2. **Integration with Existing App** (`app.py`)
- **New Mode**: "LangGraph Workflow" option in the Streamlit app
- **UI Components**: Dedicated interface for LangGraph functionality
- **Display Functions**: Specialized result visualization

### 3. **Testing Suite** (`test_langgraph.py`)
- **Unit Tests**: Comprehensive test coverage
- **Integration Tests**: End-to-end workflow testing
- **Error Handling Tests**: Fallback mechanism validation

### 4. **Demo Script** (`demo_langgraph.py`)
- **Interactive Demo**: Hands-on LangGraph experience
- **Multiple Scenarios**: Different interview types and contexts
- **Session Management**: Complete workflow demonstration

### 5. **Enhanced Notebook** (`notebooks/langGraph_test.ipynb`)
- **Advanced Examples**: Multi-step workflows and state management
- **Interactive Features**: User-driven interview simulation
- **Error Handling**: Fallback mode demonstration

## 🚀 Key Features

### **Workflow Orchestration**
- **Multi-step Process**: Question generation → Evaluation → History tracking
- **State Management**: Persistent state across workflow steps
- **Conditional Logic**: Dynamic workflow paths based on responses

### **Memory & Persistence**
- **Conversation History**: Track all interactions
- **Session Summary**: Aggregate statistics and insights
- **State Persistence**: Maintain context across sessions

### **Error Handling**
- **Fallback Mode**: Works without OpenAI API
- **Graceful Degradation**: Continues operation on errors
- **Robust Recovery**: Automatic error recovery mechanisms

### **Integration**
- **Streamlit App**: Seamless integration with existing UI
- **Backward Compatibility**: Works with existing interview flow
- **Modular Design**: Easy to extend and customize

## 🎯 Usage Examples

### **Basic Usage**
```python
from ai_modules.langgraph_processor import create_interview_processor

processor = create_interview_processor(use_openai=True)
result = processor.run_interview_round(
    round_type="HR",
    context="Software engineer",
    candidate_answer="I believe in teamwork and communication."
)
```

### **Advanced Usage**
```python
from ai_modules.langgraph_processor import InterviewState

custom_state = InterviewState(
    round_type="Technical",
    context="Senior developer",
    metadata={"level": "senior", "focus": "backend"}
)

result = processor.run_interview_round(
    round_type=custom_state.round_type,
    context=custom_state.context,
    candidate_answer="I would use microservices architecture..."
)
```

## 🔧 Configuration

### **Environment Variables**
- `OPENAI_API_KEY`: Required for full functionality
- Falls back to basic mode if not available

### **Model Configuration**
- **Default Model**: `gpt-4o-mini`
- **Temperature**: `0.7` for balanced creativity/consistency
- **Customizable**: Easy to modify model parameters

## 📊 Performance Metrics

### **Session Tracking**
- Total interactions
- Average scores
- Round type distribution
- Memory status

### **Workflow Metrics**
- Success rate
- Processing time
- Error frequency
- Fallback usage

## 🎉 Next Steps

1. **Run the Demo**: Execute `python demo_langgraph.py`
2. **Test the App**: Use "LangGraph Workflow" mode in Streamlit
3. **Explore Examples**: Run cells in `langGraph_test.ipynb`
4. **Run Tests**: Execute `python test_langgraph.py`

The LangGraph implementation is now fully integrated and ready for use! 🚀
